In [1]:
from pyspark.sql import SparkSession

# Define Iceberg version for clarity, ensure it's compatible with Spark 3.3
iceberg_version = "1.4.2" # Example: Use a recent, compatible version

spark = SparkSession.builder \
    .appName("Iceberg Local Test") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:{iceberg_version},org.apache.iceberg:iceberg-aws-bundle:{iceberg_version}") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "s3a://iceberg-warehouse/") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-89c3e2db-1d87-4368-8e99-0fed08d26ff7;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.4.2 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.4.2 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.3_2.12/1.4.2/iceberg-spark-runtime-3.3_2.12-1.4.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.4.2!iceberg-spark-runtime-3.3_2.12.jar (5895ms)
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-aws-bundle/1.4.2/iceberg-aws-bundle-1.4.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-aws-bundle;1.4.2!iceberg-aws-bundle.jar (4661ms)
:: resolution report :: resolve 1955ms 

25/06/15 11:38:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark.sql("""
CREATE TABLE local.default.customers (
    id BIGINT,
    name STRING,
    city STRING
)
USING iceberg
""")


25/06/15 11:39:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[]

In [3]:
spark.sql("""
INSERT INTO local.default.customers VALUES
    (1, 'Ali', 'Sydney'),
    (2, 'Maryam', 'Melbourne')
""")


DataFrame[]

In [4]:
spark.sql("SELECT * FROM local.default.customers").show()


+---+------+---------+
| id|  name|     city|
+---+------+---------+
|  1|   Ali|   Sydney|
|  2|Maryam|Melbourne|
+---+------+---------+



In [6]:
# List snapshots
spark.sql("SELECT * FROM local.default.customers.snapshots").show()



+--------------------+-------------------+---------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+---------+---------+--------------------+--------------------+
|2025-06-15 11:41:...|3234676519822869209|     null|   append|s3a://iceberg-war...|{spark.app.id -> ...|
+--------------------+-------------------+---------+---------+--------------------+--------------------+



In [8]:
spark.sql("CALL local.system.rollback_to_snapshot('default.customers', 3234676519822869209)")


DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]